In [8]:
import numpy as np
import pandas as pd
from stratmanager import StrategyManager
import sklearn.mixture as mix
from ta.momentum import RSIIndicator
import matplotlib.pyplot as plt

In [9]:
start_date = "2017-01-01"
end_date = "2024-02-14"
symbol = "BTC"
strat_mgr = StrategyManager(symbol, start_date, end_date)
df = strat_mgr.df.copy()
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Volume,Returns,Range,Bench_C_Rets
Date,,,,,,,,
2020-10-02,100.000000,100.000000,100.000000,100.000000,0,0.000000,0.000000,0.000300
2020-10-05,100.029999,100.040001,99.995003,99.995003,10725,-0.000050,0.000450,0.000400
2020-10-06,100.010002,100.029999,100.010002,100.029999,10080,0.000350,0.000200,-0.000050
2020-10-07,99.949997,99.959999,99.919998,99.919998,6350,-0.001100,0.000400,-0.000450
2020-10-08,99.989998,99.995003,99.971703,99.995003,650,0.000751,0.000233,0.000050
...,...,...,...,...,...,...,...,...
2024-02-06,92.180000,92.220001,92.169998,92.169998,1046,0.003539,0.000543,-0.138901
2024-02-07,92.209999,92.260002,92.120003,92.203102,703,0.000359,0.001520,-0.139462
2024-02-08,92.110001,92.180000,92.089996,92.139999,9103,-0.000684,0.000977,-0.142664


In [10]:
df_fe = df.copy()

In [11]:
rsi = RSIIndicator(close=df_fe["Close"], window=14).rsi()
df_fe["RSI"] = rsi
df_fe["RSI_Ret"] = df_fe["RSI"] / df_fe["RSI"].shift(1)

In [12]:
df_fe["MA_12"] = df_fe["Close"].rolling(window=12).mean()
df_fe["MA_21"] = df_fe["Close"].rolling(window=21).mean()

In [13]:
df_fe["DOW"] = df_fe.index.dayofweek

In [14]:
df_fe["Roll_Rets"] = df_fe["Returns"].rolling(window=30).sum()

In [15]:
df_fe["Avg_Range"] = df_fe["Range"].rolling(window=30).mean()

In [16]:
t_steps = [1, 2]
t_features = ["Returns", "Range", "RSI_Ret"]
for ts in t_steps:
    for tf in t_features:
        df_fe[f"{tf}_T{ts}"] = df_fe[tf].shift(ts)

In [17]:
df_fs = df_fe.copy()
df_fs[["Open", "High", "Low", "Volume"]] = df_fs[["Open", "High", "Low", "Volume"]].pct_change()
df_fs

,Open,High,Low,Close,Volume,Returns,Range,Bench_C_Rets,RSI,RSI_Ret,...,MA_21,DOW,Roll_Rets,Avg_Range,Returns_T1,Range_T1,RSI_Ret_T1,Returns_T2,Range_T2,RSI_Ret_T2
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-02,NaN,NaN,NaN,100.000000,NaN,0.000000,0.000000,0.000300,NaN,NaN,...,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-05,0.000300,0.000400,-0.000050,99.995003,inf,-0.000050,0.000450,0.000400,NaN,NaN,...,NaN,0,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
2020-10-06,-0.000200,-0.000100,0.000150,100.029999,-0.060140,0.000350,0.000200,-0.000050,NaN,NaN,...,NaN,1,NaN,NaN,-0.000050,0.000450,NaN,0.000000,0.000000,NaN
2020-10-07,-0.000600,-0.000700,-0.000900,99.919998,-0.370040,-0.001100,0.000400,-0.000450,NaN,NaN,...,NaN,2,NaN,NaN,0.000350,0.000200,NaN,-0.000050,0.000450,NaN
2020-10-08,0.000400,0.000350,0.000517,99.995003,-0.897638,0.000751,0.000233,0.000050,NaN,NaN,...,NaN,3,NaN,NaN,-0.001100,0.000400,NaN,0.000350,0.000200,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-06,0.003265,0.003482,0.003539,92.169998,1.789333,0.003539,0.000543,-0.138901,51.114012,1.120202,...,92.117395,1,-0.007311,0.000555,-0.004472,0.000599,0.867739,-0.006068,0.000000,0.806805
2024-02-07,0.000325,0.000434,-0.000542,92.203102,-0.327916,0.000359,0.001520,-0.139462,51.649045,1.010467,...,92.127110,2,-0.007098,0.000571,0.003539,0.000543,1.120202,-0.004472,0.000599,0.867739
2024-02-08,-0.001084,-0.000867,-0.000326,92.139999,11.948791,-0.000684,0.000977,-0.142664,50.514134,0.978027,...,92.133910,3,-0.007755,0.000604,0.000359,0.001520,1.010467,0.003539,0.000543,1.120202


In [18]:
df_fs.dropna(inplace=True)
print(df_fs.isnull().values.any())
dfobj = df_fs.isin([np.inf, -np.inf])
count = np.isinf(dfobj).values.sum()
count

False


0

In [19]:
df_fs.to_csv(f"data/{symbol}.csv")